In [2]:
from os import listdir
from re import search
# import numpy as np

DIR = "./data/"

findings = {
  31: 'D',
  41: 'A',
  66: 'A', #
  83: 'F',
  113: 'N', #
  353: 'H',
  426: 'S',
  679: 'V' #
}
indices = list(findings.keys())

seqs = []

for aff in [0, 1]:
  for filename in listdir(DIR + str(aff)):
    with open("%s%d/%s" % (DIR, aff, filename)) as f:
      lines = f.readlines()
      spcz = search(r'\[(.*?)\]',lines[0][1:]).group(1)
      name = filename.split('.')[0]
      sequence = ''.join(map(lambda k: k.strip(), lines[1:])).strip()
      seqs.append((name, spcz, sequence, aff))

def print_influence_pts(seq, name, spcz, aff, cls):
  p_str = "  "
  for i in indices: p_str += seq[i-1] + "   "
  p_str += "[%s/%s] %s (%s)" % (aff, cls, name, spcz)
  print(p_str)

def predict(seq):
  for (i, m) in list(findings.items()):
    if seq[i-1] == m: return 0
  return 1

def print_seqs(all=False):
  print(''.join(["%4d" % (i) for i in indices]), " [A/P]")
  for (name, spcz, seq, aff) in seqs: 
    if (all or aff != predict(seq)):
      print_influence_pts(seq, name, spcz, aff, predict(seq)) 
  print()

def print_all():
  print_seqs(True)

def confusion_matrix(clsfr, output=False):
  confusion = [[0, 0], [0, 0]]
  for (_, _, seq, aff) in seqs:
    cls = clsfr(seq)
    confusion[1-aff][1-cls] += 1
  if output:
    print(confusion[0])
    print(confusion[1])
  return confusion


def cluster(output=False):
  clusters = {}
  for (name, _, seq, _) in seqs:
    id = ''.join([seq[i-1] for i in indices])
    clusters[id] = [name] if not id in clusters else clusters[id] + [name]
  if output:
    print(len(clusters), "identified")
    for (id, items) in clusters.items():
      print("%s: %s" % (id, ', '.join(items)))

  return clusters


confusion_matrix(predict, True)
# print_seqs()
# cluster(True)

a = 1

# before
# [20, 3], [3, 9]
# meerkat, kangaroo rat, camel, dog, pangolin, hamster

# for s in seqs:
#   print()

print_all()

# original
# [20, 3]
# [3, 9]
# 29/35
# without 83:
# [22, 1]
# [8, 4]
# 26/35
# with 83 and 31:
# [22, 1]
# [6, 6]
# 28/35

[24, 3]
[4, 10]
  31  41  66  83 113 353 426 679  [A/P]
  T   Y   E   F   S   K   S   I   [0/0] elephant (Loxodonta africana)
  N   Y   A   Y   R   K   P   I   [0/0] raccoon (Procyon lotor)
  D   Y   G   Y   I   N   K   V   [0/0] hedgehog (Erinaceus europaeus)
  N   Y   A   F   N   H   S   V   [0/0] house-mouse (Mus musculus)
  Q   Y   G   Y   G   K   P   I   [0/1] meerkat (Suricata suricatta)
  K   Y   R   Y   S   K   P   I   [0/1] pig (Sus scrofa domesticus)
  N   Y   G   Y   S   K   P   I   [0/1] kangaroo-rat (Dipodomys ordii)
  E   Y   G   F   Y   K   P   V   [0/0] duck (Anas platyrhynchos)
  Q   Y   G   F   Q   K   P   I   [0/0] platypus (Ornithorhynchus anatinus)
  K   Y   A   F   N   H   S   V   [0/0] norway-rat (Rattus norvegicus)
  E   Y   G   Y   N   K   P   V   [0/0] guinea-pig (Cavia porcellus)
  D   H   G   F   S   K   S   I   [0/0] greater-horseshoe-bat (Rhinolophus ferrumequinum)
  K   Y   G   Y   N   R   P   I   [0/0] raccoon-dog (Nyctereutes procyonoides)
  E   Y   R  

In [7]:
from sklearn.feature_selection import SelectKBest, chi2

TRAIN, TEST = 0, 1
SEED = 1234
ALPHA = 0.025

seqs_baseline = list(filter(lambda k: k[0] not in ['pig', 'duck', 'common-marmoset', 'cynomolgus-macaque', 'fruit-bat', 'green-monkey', 'human', 'horseshoe-bat'], seqs))


def get_data():
  atts = list(map(lambda k: list(map(lambda c: ord(c), k[2])), seqs_baseline))
  label = list(map(lambda k: k[3], seqs_baseline))
  return (atts, label)

(atts, label) = get_data()
fs = SelectKBest(score_func=chi2, k='all')
fs.fit(atts, label)

p_vals = list(fs.pvalues_)
for i in range(805):
  if (p_vals[i] <= ALPHA): print('acid %d: %f' % (i+1, p_vals[i]))


acid 83: 0.018404
acid 211: 0.009965
acid 246: 0.003644
acid 251: 0.002226
acid 687: 0.000517


In [8]:
indices = [82, 210, 211, 245, 250, 686]

for ix in indices:
  res = {}
  for (name, _, seq, aff) in seqs: res[seq[ix]] = [0, 0]
  for (name, _, seq, aff) in seqs: 
    res[seq[ix]][aff] = res[seq[ix]][aff] + 1
  print(ix, res)
# all these are zero-based indexing
# 82: Y -> F
# 210: G/W/E -> D
# 211: V/E/A/I/T/S -> M/D
# 245: A/T/S -> R/S
# 250: A/T/I/N/S -> V
# 686: A/S/L -> E/V/Q/M

# all these are zero-based indexing
# new, no ones unless all ones then maybe opposite:
# 82: Y -> F [good]
# 210: G/W/E -> D [one]
# 211: V/E/A/I/T/S -> M/D/S [one, so include S]
# 245: A/T/S -> R [good]
# 250: A/T/I/N/S -> V [good]
# 686: A/S -> other [opposite, so include L]

findings = {
  82: ['F'],
  210: ['D'],
  211: ['M', 'D', 'S'],
  245: ['R'],
  250: ['V'],
  686: ['-', 'L', 'E', 'V', 'Q', 'M']
}

def predict(seq):
  for (i, acids) in findings.items():
    for a in acids: 
      if seq[i] == a: return 0
  return 1 

confusion = [[0, 0], [0, 0]]
for (name, _, seq, aff) in seqs:
  cls = predict(seq)
  confusion[1-aff][1-cls] += 1
  if (cls != aff): 
    print(name, aff)
print(confusion)

82 {'F': [6, 0], 'Y': [8, 26], 'H': [0, 1]}
210 {'-': [2, 0], 'W': [3, 10], 'G': [7, 15], 'Y': [1, 0], 'D': [1, 0], 'E': [0, 2]}
211 {'A': [6, 7], 'D': [1, 0], 'T': [1, 6], 'P': [1, 0], 'E': [2, 1], 'V': [1, 10], 'M': [1, 0], 'S': [1, 1], 'I': [0, 1], 'G': [0, 1]}
245 {'R': [4, 0], 'A': [4, 21], 'H': [1, 0], 'S': [1, 0], 'T': [4, 6]}
250 {'V': [3, 0], 'T': [7, 7], 'A': [2, 16], 'I': [1, 1], 'K': [1, 0], 'S': [0, 2], 'N': [0, 1]}
686 {'V': [1, 0], 'S': [9, 9], 'M': [2, 0], 'Q': [1, 0], 'E': [1, 0], 'A': [0, 16], '-': [0, 1], 'L': [0, 1]}
raccoon 0
pig 0
raccoon-dog 0
horseshoe-bat 1
white-tailed-deer 1
pangolin 1
[[24, 3], [3, 11]]


In [9]:
def predict(seq, fnds):
  for (i, acids) in fnds.items():
    for a in acids: 
      if seq[i-1] == a: return 0
  return 1 

news = ["pig", "duck", "civet"]

def print_confusion(res):
  modelname, fnds, data = res
  confusion = [[0, 0], [0, 0]]
  for (name, _, seq, aff) in data:
    cls = predict(seq, fnds)
    confusion[1-aff][1-cls] += 1
  acc = 100 * (confusion[0][0]+confusion[1][1])/len(data)
  sens = 100 * (confusion[0][0])/(confusion[0][0] + confusion[0][1])
  spec = 100 * (confusion[1][1])/(confusion[1][1] + confusion[1][0])
  print(modelname, acc)
  print("%.2f %.2f %.2f"%(acc, sens, spec))
  print(confusion[0])
  print(confusion[1])
  print()

# baseline model tested on 8 seqs (2 neg/6 pos) that were not used for training:
data_baseline = list(filter(lambda k: k[0] in ['pig', 'duck', 'common-marmoset', 'cynomolgus-macaque', 'fruit-bat', 'green-monkey', 'human', 'horseshoe-bat'], seqs))
# eliminative model tested on 26 seqs (9 neg/17 pos) not used for original study
data_original = list(filter(lambda k: k[0] not in ['cat', 'raccoon', 'greater-horseshoe-bat', 'mink', 'ferret', 'rabbit', 'horseshoe-bat', 'white-tailed-deer', 'siberian-tiger', 'norway-rat', 'house-mouse', 'pig', 'common-marmoset', 'cynomolgus-macaque', 'fruit-bat', 'green-monkey', 'human'], seqs))
# structural model tested on all 41 seqs (14 neg/27 pos)
data_structural = seqs
# data_structural = data_original
print(len(data_baseline), len(data_original), len(data_structural))

baseline = ['baseline', {
  83: ['F'],
  211: ['D'],
  212: ['M', 'D', 'S'],
  246: ['R'],
  251: ['V'],
  687: ['-', 'L', 'E', 'V', 'Q', 'M']
}, data_baseline]

eliminative = ['eliminative', {
  31: ['D'],
  41: ['A'],
  66: ['A'], #
  83: ['F'],
  113: ['N'], #
  353: ['H'],
  426: ['S'],
  679: ['V'] #
}, data_original]

structural = ['structural', {
  31: ['D'],
  # 41: ['A'],
  83: ['F'],
  113: ['N'], #
  353: ['H'],
  426: ['S'],
}, data_structural]

model4 = ['reduced structural', {
  31: ['D'],
  83: ['F'],
  353: ['H'],
  426: ['S'],
}, data_structural]

print_confusion(baseline)
print_confusion(eliminative)
print_confusion(structural)

# -1 pos = pig
# new neg: pig, duck (aligned)
# new pos (5/6): 
# common marmoset, macaque, fruit bat, green monkey, human (from original)
# 41 sequences total.


8 26 41
baseline 75.0
75.00 83.33 50.00
[5, 1]
[1, 1]

eliminative 76.92307692307692
76.92 82.35 66.67
[14, 3]
[3, 6]

structural 80.48780487804878
80.49 88.89 64.29
[24, 3]
[5, 9]



In [10]:
def print_misclass_pts(seq, name, spcz, aff, clss, fnds):
  indices = fnds.keys()
  p_str = "  "
  for i in indices: p_str += (seq[i-1] if seq[i-1] in fnds[i] else '*') + "   "
  p_str += "[%s/%s] %s (%s)" % (aff, cls, name, spcz)
  print(p_str)

def print_misclass(res):
  modelname, fnds, data = res
  print(modelname)
  indices = fnds.keys()
  print(''.join(["%4d" % (i) for i in indices]), " [A/P]")
  for (name, spcz, seq, aff) in data: 
    if (aff != predict(seq, fnds)):
      print_misclass_pts(seq, name, spcz, aff, predict(seq, fnds), fnds) 
  print()

print_misclass(baseline)
print_misclass(eliminative)
print_misclass(structural)
print_misclass(model4)


baseline
  83 211 212 246 251 687  [A/P]
  *   *   *   *   *   *   [0/1] pig (Sus scrofa domesticus)
  *   *   S   *   *   *   [1/1] horseshoe-bat (Rhinolophus macrotis)

eliminative
  31  41  66  83 113 353 426 679  [A/P]
  *   *   *   *   *   *   *   *   [0/1] meerkat (Suricata suricatta)
  *   *   *   *   *   *   *   *   [0/1] kangaroo-rat (Dipodomys ordii)
  *   *   *   *   *   *   *   *   [0/1] camel (Camelus dromedarius)
  *   *   *   *   N   *   *   *   [1/1] domestic-dog (Canis lupus familiaris)
  *   *   *   *   N   *   *   V   [1/1] pangolin (Manis pentadactyla)
  *   *   A   *   N   *   S   V   [1/1] hamster (Mesocricetus auratus)

structural
  31  83 113 353 426  [A/P]
  *   *   *   *   *   [0/1] raccoon (Procyon lotor)
  *   *   *   *   *   [0/1] meerkat (Suricata suricatta)
  *   *   *   *   *   [0/1] pig (Sus scrofa domesticus)
  *   *   *   *   *   [0/1] kangaroo-rat (Dipodomys ordii)
  *   *   *   *   *   [0/1] camel (Camelus dromedarius)
  *   *   N   *   *   [1/1] do

In [11]:
def mcnemar(fnds1, fnds2):
  matrix = [[0,0],[0,0]]
  for (_, _, seq, _) in seqs:
    matrix[1-predict(seq, fnds1)][1-predict(seq, fnds2)] += 1
  return matrix

def mcnemar_print(m1, m2):
  name1, fnds1, _ = m1
  name2, fnds2, _ = m2
  print("mcnemar: %s vs %s" % (name1, name2))
  matrix = mcnemar(fnds1, fnds2)
  print(matrix[0])
  print(matrix[1])

mcnemar_print(baseline, structural)
mcnemar_print(baseline, eliminative)
mcnemar_print(eliminative, structural)
mcnemar_print(structural, model4)
# print("mcnemar(baseline[1], structural[1]))

mcnemar: baseline vs structural
[24, 3]
[5, 9]
mcnemar: baseline vs eliminative
[23, 4]
[5, 9]
mcnemar: eliminative vs structural
[28, 0]
[1, 12]
mcnemar: structural vs reduced structural
[29, 0]
[4, 8]


In [20]:
pred_seqs = []

for filename in listdir(DIR + "2"):
  with open("%s%d/%s" % (DIR, 2, filename)) as f:
    lines = f.readlines()
    spcz = search(r'\[(.*?)\]',lines[0][1:]).group(1)
    name = filename.split('.')[0]
    sequence = ''.join(map(lambda k: k.strip(), lines[1:])).strip()
    pred_seqs.append((name, spcz, sequence))

def pred(s):
  name, _, seq = s
  for model in [baseline, eliminative, structural]:
    fnds = model[1]
    print(predict(seq, fnds), end=" ")
  print(name)

print("B E S species")
for s in pred_seqs: pred(s)

B E S species
0 1 1 black-flying-fox
1 1 1 sumatran-orangutan
1 0 0 chinchilla
1 1 1 grizzly-bear
0 0 0 turkey
0 0 0 tasmanian-devil
0 0 0 leatherback-sea-turtle
1 1 1 orca
